In [117]:
import numpy as np
from scipy.integrate import quad

def f(x):
    return np.log(1+np.tan(x))
    
xs = 0
xe = np.pi/4

def trap(f,xs,xe,npanel):
    a = np.linspace(xs,xe,2**(npanel-1)+1)
    h = np.diff(a)[0]
    return (2*sum(f(a))-f(a[0])-f(a[-1])) * h/2
R11 = trap(f,xs,xe,1)

R21 = trap(f,xs,xe,2)
R22 = (2**2 * R21 - R11) / (2**2 - 1) 

R31 = trap(f,xs,xe,3)
R32 = (2**2 * R31 - R21) / (2**2 - 1)
R33 = (2**4 * R32 - R22) / (2**4 - 1)

R41 = trap(f,xs,xe,4)
R42 = (2**2 * R41 - R31) / (2**2 - 1)
R43 = (2**4 * R42 - R32) / (2**4 - 1)
R44 = (2**8 * R43 - R33) / (2**8 - 1)

print(f'''
R11             | {R11}
R21 R22         | {R21}  {R22}
R31 R32 R33     | {R31}  {R32}  {R33}
R41 R42 R43 R44 | {R41} {R42} {R43} {R44}
Converged Integral Value : {quad(f,0,np.pi/4)[0]}
''')



R11             | 0.27219826128795027
R21 R22         | 0.2721982612879502  0.27219826128795016
R31 R32 R33     | 0.2721982612879502  0.2721982612879502  0.2721982612879502
R41 R42 R43 R44 | 0.27219826128795027 0.27219826128795027 0.27219826128795027 0.27219826128795027
Converged Integral Value : 0.27219826128795027



In [114]:
# trapezoid
x = np.linspace(0,np.pi,5)
fx = np.array([1.0,0.3431,0.25,0.3431,1.0])
h = np.diff(x)[0]
trapezoid = (2*sum(fx)-fx[0]-fx[-1])*h/2

# simpson
def simp(y,h):
    I1 = h*(y[0] + y[-1])
    I4 = 4*h*(sum(y[1::2]))
    I2 = 2*h*sum(y[2:-2:2])
    Itot = (I1 + I4 + I2)/3
    return Itot

# romberg
R11 = (fx[0] + fx[4]) * (x[4] - x[0])/2

R21 = (fx[0] + 2*fx[2] + fx[4]) * (x[2] - x[0])/2
R22 = (2**2 * R21 - R11) / (2**2 - 1) 

R31 = (fx[0] + 2*fx[1] + 2*fx[2] + 2*fx[3] + fx[4]) * (x[1] - x[0])/2
R32 = (2**2 * R31 - R21) / (2**2 - 1)
R33 = (2**4 * R32 - R22) / (2**4 - 1)

print(f'trapezoid : {trapezoid} + O(h^2)\nsimpson   : {simp(fx,h)} + O(h^4)\nromberg   : {R33} + O(h^6)')
print('이 경우에는 romberg방법의 오차의 계수가 적으므로 romberg의 정확도가 가장 높다.')

trapezoid : 1.5206879239701392 + O(h^2)
simpson   : 1.3730854291289791 + O(h^4)
romberg   : 1.3599047026179174 + O(h^6)
이 경우에는 romberg방법의 오차의 계수가 적으므로 romberg의 정확도가 가장 높다.


In [105]:
import numpy as np
from scipy.integrate import quad
def gaussNodes(m,tol=10e-9):

    def legendre(t,m):
        p0 = 1.0; p1 = t
        for k in range(1,m):
            p = ((2.0*k + 1.0)*t*p1 - k*p0)/(1.0 + k )
            p0 = p1; p1 = p
        dp = m*(p0 - t*p1)/(1.0 - t**2)
        return p,dp
    A = np.zeros(m)
    x = np.zeros(m)
    nRoots = int((m + 1)/2) # Number of non-neg. roots
    for i in range(nRoots):
        t = np.cos(np.pi*(i + 0.75)/(m + 0.5))# Approx. root
        for j in range(30):
            p,dp = legendre(t,m) # Newton-Raphson
            dt = -p/dp; t = t + dt # method
            if abs(dt) < tol:
                x[i] = t; x[m-i-1] = -t
                A[i] = 2.0/(1.0 - t**2)/(dp**2) # Eq.(6.25)
                A[m-i-1] = A[i]
                break
    return x,A

def gaussQuad(f,a,b,m):
    c1 = (b + a)/2.0
    c2 = (b - a)/2.0
    x,A = gaussNodes(m)
    sum = 0.0
    for i in range(len(x)):
        sum = sum + A[i]*f(c1 + c2*x[i])
    return c2*sum
def f(x):    return np.log(x)/(x**2 - 2*x + 2)

print(f'''
gauss_Legendre sol nodes = 2 : {gaussQuad(f,1,np.pi,2)}
gauss_Legendre sol nodes = 4 : {gaussQuad(f,1,np.pi,4)} 
Converged Integral Value     : {quad(f,1,np.pi)[0]}
''')


gauss_Legendre sol nodes = 2 : 0.6067250072484867
gauss_Legendre sol nodes = 4 : 0.5847680362120717 
Converged Integral Value     : 0.5849428069312876

